## 先安装依赖：

pip install alibabacloud_devops20210625

In [8]:
# pip install --index-url=https://mirrors.aliyun.com/pypi/simple/ --trusted-host=mirrors.aliyun.com --default-timeout=30 alibabacloud_devops20210625

## 配置ALI云账户信息

In [10]:
import os

from typing import List

from alibabacloud_devops20210625.client import Client as devops20210625Client
from alibabacloud_tea_openapi import models as open_api_models
from alibabacloud_tea_util import models as util_models
from alibabacloud_tea_util.client import Client as UtilClient
from alibabacloud_devops20210625 import models as devops_20210625_models
from alibabacloud_tea_util.models import RuntimeOptions
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

# Mac用户可以通过zsh 或者 bash来设置全局环境变量：
# 比如：
# echo "export ALIBABA_CLOUD_ACCESS_KEY_ID=ABCD1234" >> ~/.zshrc && source ~/.zshrc

# 需要先获取 阿里云的access key： https://ram.console.aliyun.com/manage/ak
# 如果没有，则需要新建一对key 和secret

ALIBABA_CLOUD_ACCESS_KEY_ID = os.environ.get(
    "ALIBABA_CLOUD_ACCESS_KEY_ID", "LTAI5t5hyq7WAPwVc516iY9e"
)  # 也可以直接把ACCESS_KEY_ID放在这里

ALIBABA_CLOUD_ACCESS_KEY_SECRET = os.environ.get(
    "ALIBABA_CLOUD_ACCESS_KEY_SECRET", "VPuyCowWOSxsoLDuakjKt3mGzwm4rr"
)  # 也可以直接把ACCESS_KEY_SECRET放在这里

# 配置阿里云SDK：
config = open_api_models.Config(
    access_key_id=ALIBABA_CLOUD_ACCESS_KEY_ID,  # 请在阿里云上创建自己的ACCESS_KEY
    access_key_secret=ALIBABA_CLOUD_ACCESS_KEY_SECRET,  # 请在阿里云上创建自己的ACCESS_KEY_SECRET
)
# Endpoint 请参考 https://api.aliyun.com/product/devops
config.endpoint = f"devops.cn-hangzhou.aliyuncs.com"
client = devops20210625Client(config)

organization_id = "6189f099041d450d2c253abc"  # 这里固定值，为公司在云效上的唯一ID

project_id = "a41d2acc436b9755f11b79d3d4"  # 云效项目ID。这里每个组（供应链、SaaS/商城）不一样，可以通过云效果链接获取到，比如:
# https://devops.aliyun.com/projex/project/a41d2acc436b9755f11b79d3d4/task#viewIdentifier=6c6e9be7f1bafb355bff57fefa
# 以上链接中project/a41d2acc436b9755f11b79d3d4/task 即可拿到project_id；

# 先获取当前用户的名字：
res=client.get_user_info(request=devops_20210625_models.GetUserInfoRequest(organization_id=organization_id))
my_name=res.body.to_map()['result']['name']
logging.info(f"你的大名:{my_name}")

# 获取用户的ID：
res=client.list_organization_members(organization_id=organization_id, request=devops_20210625_models.ListOrganizationMembersRequest(organization_member_name=my_name))
members=res.body.to_map()
if len(members)<=0 or 'members' not in members:
    raise Exception(f"找不到用户:{my_name}")
else:
    members=members['members'][0]
    my_aliyun_pk=members['accountId']
    logging.info(f"当前用户:{members}")

logging.info(f"{ALIBABA_CLOUD_ACCESS_KEY_ID[0:10]}, {ALIBABA_CLOUD_ACCESS_KEY_SECRET[0:10]}, {my_aliyun_pk}")

2024-06-26 10:59:05 - INFO - 你的大名:唐鹏
2024-06-26 10:59:05 - INFO - 当前用户:{'accountId': '213214975667067024', 'deptLists': ['商城&SaaS后端'], 'email': '', 'identities': {'externUid': '', 'provider': ''}, 'joinTime': 1675671700404, 'lastVisitTime': 1719368846974, 'mobile': '', 'organizationMemberName': '唐鹏', 'organizationRoleId': '62c5099ac8be86003f833b6e', 'organizationRoleName': '运维', 'state': 'normal'}
2024-06-26 10:59:05 - INFO - LTAI5tQzmp, EhaF73Jp1Q, 213214975667067024


In [21]:
runtime = RuntimeOptions()
headers = {"Content-Type": "application/json"}


def search_workitem_by_name(work_item_name="manage"):
    conditionGroups = [
        [
            {
                "fieldIdentifier": "assignedTo",
                "operator": "CONTAINS",
                "value": [my_aliyun_pk],
                "toValue": None,
                "className": "user",
                "format": "list",
            },
            # {
            #     "fieldIdentifier": "subject",
            #     "operator": "CONTAINS",
            #     "value": [work_item_name],
            #     "toValue": None,
            #     "className": "string",
            #     "format": "input",
            # },
        ]
    ]
    conditions = {"conditionGroups": conditionGroups}
    req = devops_20210625_models.ListWorkitemsRequest(
        space_type="Project",
        space_identifier="a41d2acc436b9755f11b79d3d4",
        category="Task",
        max_results="50",
        # conditions=conditions,
        group_condition=conditionGroups,
        search_type="LIST",
    )
    # 查询自己最近的任务是否有已存在的
    res = client.list_workitems_with_options(
        request=req, organization_id=organization_id, runtime=runtime, headers=headers
    )
    query_error_code = res.body.to_map()["errorCode"]
    if query_error_code != "success":
        return None
    work_items = res.body.to_map()["workitems"]
    return work_items


for item in search_workitem_by_name("周会"):
    logging.info(f"{item['serialNumber']},{item['subject']},{item['assignedTo']}")

## 创建任务并且录入工时

- 任务仅可用作日常任务的创建；
- 项目任务仍然需要在对应的《需求》下创建并及时录入工时

In [ ]:
from datetime import datetime
import json


gmt_start = datetime.now().strftime("%Y-%m-%d")
worktime = "1"
time_type = {
    "研发": "develop",
    "测试": "test",
    "其他": "others",
    "联调": "85d41d4298fe223ae4313c2cd4",
    "冒烟": "e5060c97d239c8a957642deac8",
    "设计": "3a19b39714b0d0cd86ffc95687",
}


def create_task_for_me(task={}, gmt_start=gmt_start):
    subject = task["任务名"]
    worktime = task["工时"]
    task_type = "SaaS"
    if "归属业务" in task:
        logging.info(f"设置单独的Task Type:{task}")
        task_type = task["归属业务"]
    sprint_id = "fe2afd768b09b7e8fde816a21c"  # 其他事项
    if "会议" in subject or "周会" in subject:
        logging.info(f"这是一个会议：{subject}")
        sprint_id = "019dc37e2d25e42d6b5c6cd81e"  # 各种会议

    logging.info(f"===\n将为你创建任务：{subject}")

    field_value_list = []
    field_value_list.append(
        devops_20210625_models.CreateWorkitemV2RequestFieldValueList(
            field_identifier="fba0a046ffed2f9c5224db5493", value="SaaS"
        )
    )  # 业务模块
    field_value_list.append(
        devops_20210625_models.CreateWorkitemV2RequestFieldValueList(
            field_identifier="79", value=f"{gmt_start} 00:00:00"
        )
    )  # 计划开始时间
    field_value_list.append(
        devops_20210625_models.CreateWorkitemV2RequestFieldValueList(
            field_identifier="80", value=f"{gmt_start} 00:00:00"
        )
    )  # 计划完成时间
    # field_value_list.append(devops_20210625_models.CreateWorkitemV2RequestFieldValueList(field_identifier='0c906d95f677f4bd1c5d75e4a7',value="技术类需求")) # "技术类需求"
    field_value_list.append(
        devops_20210625_models.CreateWorkitemV2RequestFieldValueList(
            field_identifier="101586", value=worktime
        )
    )  # 预估工时
    field_value_list.append(
        devops_20210625_models.CreateWorkitemV2RequestFieldValueList(
            field_identifier="priority", value="e5d3d463361fe45fafc01a3b8a"
        )
    )  # 优先级

    # 创建一个新的空的任务
    req = devops_20210625_models.CreateWorkitemV2Request(
        assigned_to=my_name,
        category="Task",
        description=f"自动创建于:{datetime.now()}",
        field_value_list=field_value_list,
        space_identifier="a41d2acc436b9755f11b79d3d4",  # 项目空间
        sprint_identifier=sprint_id,  # 迭代ID
        subject=f"{subject}-{gmt_start}",
        # tags=["测试"],
        workitem_type_identifier="ba102e46bc6a8483d9b7f25c",
    )

    res = client.create_workitem_v2(request=req, organization_id=organization_id)
    new_item = res.body.to_map()
    pretty_output = json.dumps(new_item, indent=4)
    logging.info(f"创建新任务成功:\n{pretty_output}")

    work_item_id = new_item["workitemIdentifier"]

    # 更新任务的计划开始时间
    req = devops_20210625_models.UpdateWorkItemRequest(
        identifier=work_item_id,
        property_key="79",
        property_value=f"{gmt_start} 00:00:00",
        field_type="customField",
    )
    res = client.update_work_item(request=req, organization_id=organization_id)
    logging.info(f"更新任务的计划开始时间 成功:{res.body.to_map()['errorCode']}")

    # 更新任务的计划完成时间
    req = devops_20210625_models.UpdateWorkItemRequest(
        identifier=work_item_id,
        property_key="80",
        property_value=f"{gmt_start} 00:00:00",
        field_type="customField",
    )
    res = client.update_work_item(request=req, organization_id=organization_id)
    logging.info(f"更新任务的计划完成时间 成功:{res.body.to_map()['errorCode']}")

    # 更新任务的业务线
    req = devops_20210625_models.UpdateWorkItemRequest(
        identifier=work_item_id,
        property_key="fba0a046ffed2f9c5224db5493",
        property_value=task_type,
        field_type="customField",
    )
    res = client.update_work_item(request=req, organization_id=organization_id)
    logging.info(f"更新任务的业务线 成功:{res.body.to_map()['errorCode']}")

    # 更新任务的任务类型
    # req=devops_20210625_models.UpdateWorkItemRequest(identifier=work_item_id,property_key='0c906d95f677f4bd1c5d75e4a7',property_value='技术类需求',field_type='customField')
    # res=client.update_work_item(request=req,organization_id=organization_id)
    # logging.info(res.body.to_map())

    # 更新任务的优先级
    req = devops_20210625_models.UpdateWorkItemRequest(
        identifier=work_item_id,
        property_key="priority",
        property_value="e5d3d463361fe45fafc01a3b8a",
        field_type="customField",
    )
    res = client.update_work_item(request=req, organization_id=organization_id)
    logging.info(f"更新任务的优先级 成功:{res.body.to_map()['errorCode']}")

    # 新增任务的预计工时
    # req = devops_20210625_models.CreateWorkitemEstimateRequest(
    #     description=f"自动创建于:{datetime.now()}",
    #     record_user_identifier=my_aliyun_pk,
    #     spent_time=worktime,
    #     type=time_type["研发"],
    #     workitem_identifier=work_item_id,
    # )
    # res = client.create_workitem_estimate(request=req, organization_id=organization_id)
    # logging.info(res.body.to_map())

    # 新增任务的实际工时
    req = devops_20210625_models.CreateWorkitemRecordRequest(
        actual_time=worktime,
        description=f"自动创建于:{datetime.now()}",
        gmt_end=f"{gmt_start}",
        gmt_start=f"{gmt_start}",
        record_user_identifier=my_aliyun_pk,
        type=time_type["研发"],
        workitem_identifier=work_item_id,
    )
    res = client.create_workitem_record(request=req, organization_id=organization_id)
    logging.info(f"新增任务的实际工时 成功:{res.body.to_map()['errorCode']}")

    # 更新状态任务为已完成
    req = devops_20210625_models.UpdateWorkItemRequest(
        identifier=work_item_id,
        property_key="status",
        property_value="100014",
        field_type="status",
    )
    res = client.update_work_item(request=req, organization_id=organization_id)
    final_result = res.body.to_map()
    workitem=final_result['workitem']
    new_id=workitem['identifier']
    url=f"https://devops.aliyun.com/projex/project/a41d2acc436b9755f11b79d3d4/task#viewIdentifier=ab07d07c9654b95a147691f2&openWorkitemIdentifier={new_id}"
    logging.info(f"成功了: {workitem['serialNumber']}, {workitem['subject']}, {url}\n")



def examine_task_exists(work_item_subject):
    conditions = {
        "conditionGroups": [
            [
                {
                    "fieldIdentifier": "assignedTo",
                    "operator": "CONTAINS",
                    "value": [my_aliyun_pk],
                    "toValue": None,
                    "className": "user",
                    "format": "list"
                }
            ]
        ]
    }
    req = devops_20210625_models.ListWorkitemsRequest(
        space_type='Project',
        space_identifier='a41d2acc436b9755f11b79d3d4',
        category='Task',
        max_results='50',
        conditions=conditions,
        search_type='LIST',
    )
    runtime = RuntimeOptions(
    )
    headers = {
        'Content-Type': 'application/json'
    }
    # 查询自己最近的任务是否有已存在的
    res = client.list_workitems_with_options(request=req, organization_id=organization_id, runtime=runtime,
                                             headers=headers)
    query_error_code = res.body.to_map()['errorCode']
    if query_error_code != 'success':
        return None
    work_items = res.body.to_map()['workitems']
    for item in work_items:
        item['subject'] = remove_date_suffix(item['subject'])
        if item['subject'] == work_item_subject:
            work_item_create_date = transfer_unix_2_date(item['gmtCreate'])
            logging.info(f"检测到相同的任务,原任务ID:{item['identifier']},创建时间:{work_item_create_date},即将追加工时...")
            return item['identifier']


import re


def remove_date_suffix(task_name):
    # 使用正则表达式匹配以 -YYYY-MM-DD 结尾的部分
    pattern = r'-\d{4}-\d{2}-\d{2}$'
    # 替换为空字符串
    cleaned_name = re.sub(pattern, '', task_name)
    return cleaned_name.strip()


def transfer_unix_2_date(unix_time):
    # 将毫秒时间戳转换为秒（需要除以1000）
    timestamp_seconds = unix_time / 1000

    # 使用 datetime.fromtimestamp() 将时间戳转换为日期时间对象
    dt_object = datetime.fromtimestamp(timestamp_seconds)

    # 格式化日期时间为 yyyy-mm-dd 格式
    return dt_object.strftime('%Y-%m-%d')


def append_task_for_me(work_item_id, task_time):
    # 追加任务的预计工时
    req = devops_20210625_models.CreateWorkitemEstimateRequest(
        description=f"自动追加创建于:{datetime.now()}",
        record_user_identifier=my_aliyun_pk,
        spent_time=task_time,
        type=time_type["研发"],
        workitem_identifier=work_item_id,
    )
    res = client.create_workitem_estimate(request=req, organization_id=organization_id)
    logging.info(f"追加任务的预计工时 成功:{res.body.to_map()['errorCode']}")
    # 追加任务的实际工时
    req = devops_20210625_models.CreateWorkitemRecordRequest(
        actual_time=task_time,
        description=f"自动追加创建于:{datetime.now()}",
        gmt_end=f"{gmt_start}",
        gmt_start=f"{gmt_start}",
        record_user_identifier=my_aliyun_pk,
        type=time_type["研发"],
        workitem_identifier=work_item_id,
    )
    res = client.create_workitem_record(request=req, organization_id=organization_id)
    logging.info(f"追加任务的实际工时 成功:{res.body.to_map()['errorCode']}")
    
    req = devops_20210625_models.UpdateWorkItemRequest(
        identifier=work_item_id,
        property_key="80",
        property_value=f"{gmt_start} 00:00:00",
        field_type="customField",
    )
    res = client.update_work_item(request=req, organization_id=organization_id)
    logging.info(f"更新任务的计划完成时间 成功:{res.body.to_map()['errorCode']}")

## 创建任务
* **任务名**：任务名称，本程序会自动在任务名后面加上日期
* **工时**：任务工时，支持小数，比如: 1.5, 2, 3
* **归属业务**：可选项目：'技术需求', 'SaaS', '全品类', '大客户', '鲜沐商城', 'CRM', '其他'

In [ ]:
task_type = [
    "技术需求",
    "SaaS",
    "全品类",
    "大客户",
    "鲜沐商城",
    "CRM",
    "供应链计划（SCP）",
    "采购&供应商（PMS&SRM）",
    "履约&库存",
    "仓储（WMS）",
    "仓网配送（WNC&TMS）",
    "供应链费用（BMS）",
    "供应链其他相关",
    "财务相关（应收&应付）",
    "其他",
]
tasks_to_create = [
    # {'任务名':"test", '工时':'0.5','归属业务':'技术需求'},
    # {'任务名':"POP项目-分期发布排期会议", '工时':'1.5','归属业务':'技术需求'},
    {'任务名':"老manage集群化部署-离线任务移除-Java锁梳理", '工时':'3','归属业务':'技术需求'},
    {'任务名':"SonarQube增量扫描调研", '工时':'1','归属业务':'技术需求'},
    {'任务名':"老manage导出销售数据优化-任务化执行-待发布", '工时':'2','归属业务':'技术需求'},
]
for task in tasks_to_create:
    work_item_id = examine_task_exists(task["任务名"])
    if work_item_id is not None:
        # 已存在的任务 追加工时
        append_task_for_me(work_item_id=work_item_id, task_time=task["工时"])
    else:
        create_task_for_me(task)  # 创建当天的任务
    # create_task_for_me(task, gmt_start='2024-01-01') # 用来补任务（请尽力在下班前完成工时的录入）